This notebok takes row stamps as GPX data and export it to a readable Excel files using pandas


In [16]:
import pandas as pd
import os
import sys
import gpxpy.gpx
import gpxpy
from pathlib import Path
#path_to_harz = Path(os.getcwd(), 'source')
path_to_harz = Path("..")

In [17]:
def transform_stamp_name_to_int(name):
    """
    This function takes "HWN002 Scharfenstein" and should return 2
    """
    try:
        number_str = name.split("HWN")[1][:3]
    except:
        number_str = -1
    return int(number_str)

export to excel

In [18]:
import gpxpy
import gpxpy.gpx
import pandas as pd

# Open the GPX file
gpx_file = open(Path(path_to_harz, 'gpx', 'HWN2024.gpx'), 'r')
gpx = gpxpy.parse(gpx_file)

# List to hold waypoint data
waypoints_data = []

# Iterate through the waypoints and collect data
for waypoint in gpx.waypoints:
    waypoint_info = {
        'number': transform_stamp_name_to_int(waypoint.name),
        'name': waypoint.name,
        'latitude': waypoint.latitude,
        'longitude': waypoint.longitude,
        'elevation': waypoint.elevation,
        'description': waypoint.description,
        'comment': waypoint.comment,
    }
    waypoints_data.append(waypoint_info)

# Create a DataFrame
waypoints_df = pd.DataFrame(waypoints_data)

# Display the DataFrame
# print(waypoints_df)

# Optionally, save the DataFrame to a CSV file
waypoints_df.to_excel(
    Path(path_to_harz, "excel", 'HWN2024_export_gpx.xlsx'), index=False)

in the gpx folder we exported additional themed hiking pins ("Harzer Hexenstieg", "Harzer Steiger", "Harzer Grenzweg", "Goethe im Harz")

In [19]:
path_to_gpx = Path(path_to_harz, 'gpx')
all_stamps = pd.read_excel(Path(
    path_to_harz, 'excel', 'HWN2024_export_gpx.xlsx'))
hexenstieg = pd.read_excel(Path(
    path_to_gpx, 'HWN_Hexenstieg_exported_from_gpx.xlsx'))
hazer_steiger = pd.read_excel(Path(
    path_to_gpx, 'HWN_Steiger_exported_from_gpx.xlsx'))
grenzweg = pd.read_excel(Path(
    path_to_gpx, 'HWN_Grenzweg_W_exported_from_gpx.xlsx'))
goethe = pd.read_excel(Path(
    path_to_gpx, 'HWN_Goethe_exported_from_gpx.xlsx'))

# add number column to all dataframes
list_dataframes = [hexenstieg, hazer_steiger, goethe, grenzweg, ]
for df in list_dataframes:
    df['number'] = df['name'].apply(transform_stamp_name_to_int)

add this information as indexes to the end datarfame

In [20]:
all_stamps["hexe"] = None
all_stamps["steiger"] = None
all_stamps["grenze"] = None
all_stamps["goethe"] = None

for df, col_name in zip(list_dataframes, ["hexe", "steiger", "goethe", "grenze"]):
    for i, row in all_stamps.iterrows():
        if row["number"] in df["number"].tolist():
            all_stamps.loc[i, col_name] = 1
        else:
            all_stamps.loc[i, col_name] = 0

In [21]:
all_stamps

,number,name,latitude,longitude,elevation,description,comment,hexe,steiger,grenze,goethe
0,1,HWN001 Eckertalsperre,51.841649,10.579976,555,Eckertalsperre (Staumauer),Eckertalsperre (Staumauer),0,0,1,0
1,2,HWN002 Scharfenstein,51.830168,10.602774,635,Scharfenstein (Rangerstation),Scharfenstein (Rangerstation),0,0,1,0
2,3,HWN003 Am Kruzifix,51.844150,10.611610,567,Am Kruzifix (Schutzhütte),Am Kruzifix (Schutzhütte),0,0,1,0
3,4,HWN004 Taubenklippe,51.862558,10.618166,555,Taubenklippe,Taubenklippe,0,0,1,0
4,5,HWN005 Froschfelsen,51.851915,10.654570,523,Froschfelsen,Froschfelsen,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
217,218,HWN218 Neustädter Talsperre,51.578883,10.863333,460,Neustädter Talsperre,Neustädter Talsperre,0,0,0,0
218,219,HWN219 Wippertalsperre,51.569720,11.192660,300,Wippertalsperre Bank am Rundweg,Wippertalsperre Bank am Rundweg,0,0,0,0
219,220,HWN220 Phillippsgruß,51.609504,10.531842,464,Phillippsgruß,Phillippsgruß,0,0,0,0
220,221,HWN221 Jungfernklippe,51.804200,10.513817,650,Jungfernklippe,Jungfernklippe,0,0,0,0


In [22]:
all_stamps.to_excel(
    Path(path_to_harz, "excel", 'all_stamps.xlsx'), index=False)